sync facebook threads?



In [ ]:
var importer = require('../Core');
var fs = require('fs');
var runSeleniumCell = importer.import('selenium cell');
var loginFacebook, listFacebookThreads, readFacebookThread;

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var project = PROFILE_PATH + '/Conversations';

$$.async();
runSeleniumCell([
    'log in facebook',
    'list facebook threads',
    'messages from facebook',
])
    .then(r => ({
        loginFacebook, listFacebookThreads, readFacebookThread
    } = r).loginFacebook())
    .then(threads => {
        try { threads = JSON.parse(fs.readFileSync(project + '/facebook-threads.json')); }
        catch (e) { threads = []; }
    // TODO: compare glob/stat to see which threads need updating
        return importer.runAllPromises(threads.slice(450, 600)
            .map(t => ((resolve) => readFacebookThread(t).then((r) => resolve(r)))));
    })
    // TODO: when threads gets to 100% call listFacebookThreads()
    //.then(() => listFacebookThreads())


collect facebook profiles?



In [3]:
var {JSDOM} = require('jsdom');
var fs = require('fs');
var path = require('path');
var glob = require('glob');
var assert = require('assert');
var importer = require('../Core');
var runSeleniumCell = importer.import('selenium cell');
var loginFacebook,
    likeAllPosts,
    getAllXPath,
    scrapeFacebookFriends;

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var project = PROFILE_PATH + '/Conversations';

function parseFacebookFriends() {
    return getAllXPath([
        '//a[contains(@href, "friends_tab")]/@href'
    ])
    .then(friends => {
        return friends
          .filter((elem, pos, arr) => arr.indexOf(elem) === pos)
          .map(f => f.replace(/\?.*$/ig, ''))
    })
}

var FRIENDS_FILE;
function collectFacebookProfiles() {
    var fresh;
    return runSeleniumCell([
        'log in facebook',
        'like all facebook posts',
        'scrape facebook profile',
        'scrape facebook friends',
    ])
        .then(r => ({
            loginFacebook,
            likeAllPosts,
            scrapeFacebookFriends
        } = r).loginFacebook())
        // TODO: abstract this data collection from JSDOM
        // TODO: only scrape once per day? use last file instead?
        //.then(() => scrapeFacebookFriends())
        .then(() => {
            var friends = glob.sync('**/*friend*', {cwd: project, nodir: true});
            friends.sort((a, b) => 
                new Date(fs.statSync(path.join(project, a)).mtime).getTime()
                    - new Date(fs.statSync(path.join(project, b)).mtime).getTime());
            FRIENDS_FILE = path.join(project, friends.pop());
            return fs.readFileSync(FRIENDS_FILE).toString();
        })
        .then(doc => {
            // call script to get all Facebook friends
            var dom = new JSDOM(doc);
            getAllXPath = importer.import('all elements xpath',  {
                client: {
                    execute: (func, ...args) => Promise.resolve({
                        value: func.apply(dom.window.document, args)
                    }),
                    addCommand: () => {
                    }
                },
                document: dom.window.document,
                XPathResult: {ORDERED_NODE_ITERATOR_TYPE: 5}
            })
            return parseFacebookFriends();
        })
        .catch(e => console.log(e))
        .then(friends => {
            assert(friends.length > 0, FRIENDS_FILE + ' should have friends links in it')
            // use glob.sync to make sure every friend is hit at least once in a rotation
            var existingPosts = glob.sync('**/*-posts-*.json', {cwd: project});
            // TODO: way to tell which part of the URL is unique?
            var friendCount = friends.length;
            fresh = friends.filter(profile => {
                const profileId = profile.replace(/^\/|\/$/ig, '').split('/').pop()
                .replace(/[^a-z0-9]/ig, '_');
                // TODO: check for file.stat instead of year?
                return existingPosts.indexOf(profileId
                    + '-posts-' + (new Date()).getFullYear() + '.json') === -1;
            });
            const percent = Math.round((friendCount - fresh.length) / friendCount * 100);
            console.log((friendCount - fresh.length) + ' / ' + friendCount + ' : '
                + percent
                + '%');
            //return percent === 100 ? scrapeFacebookFriends() : [];
        })
        .then(() => {
            const rand = Math.round(fresh.length * Math.random());
            console.log(fresh.slice(rand, rand + 1));
            return importer.runAllPromises(fresh.slice(rand, rand + 1)
                .map(p => (resolve) => likeAllPosts(p, null).then(r => resolve(r))));
        })
        .catch(e => console.log(e))
};
module.exports = collectFacebookProfiles;


Initializing webdriver on localhost


[Function: collectFacebookProfiles]

In [1]:
var importer = require('../Core');
var collectFacebookProfiles = importer.import('collect facebook profiles')
/*

    https://www.facebook.com/dadsrawesome/videos/1165913990203850/
    https://www.facebook.com/galacticempireofficial/videos/800461353413445/
        */
$$.async();
collectFacebookProfiles()
    .then(r => $$.sendResult(r))
    .catch(e => $$.sendError(e))


Initializing webdriver on localhost
[22:14:50]  COMMAND	GET 	 "/wd/hub/session/1c8ba9c91f146851229328caa33b824c"
[22:14:50]  COMMAND	GET 	 "/wd/hub/session/187aa56fb32a9a1891ffafbc84fcd3b9"
[22:14:50]  COMMAND	GET 	 "/wd/hub/session/65e30a37d1acb20f724dcc385caa946f"
[22:14:50]  COMMAND	GET 	 "/wd/hub/session/714a4207c3a5f6445fe605a96f4a8f34"
[22:14:50]  COMMAND	GET 	 "/wd/hub/session/a59fe47b233f672a6fc9a99ada843b55"
[ '1c8ba9c91f146851229328caa33b824c',
  '187aa56fb32a9a1891ffafbc84fcd3b9',
  '65e30a37d1acb20f724dcc385caa946f',
  '714a4207c3a5f6445fe605a96f4a8f34',
  'a59fe47b233f672a6fc9a99ada843b55' ]
0
[22:14:50]  COMMAND	GET 	 "/wd/hub/session/1c8ba9c91f146851229328caa33b824c/window_handle"
[22:14:50]  COMMAND	POST 	 "/wd/hub/session/1c8ba9c91f146851229328caa33b824c/window"
[22:14:50]  COMMAND	POST 	 "/wd/hub/session/1c8ba9c91f146851229328caa33b824c/execute"
[22:14:50]  COMMAND	GET 	 "/wd/hub/session/1c8ba9c91f146851229328caa33b824c/window_handles"
[22:14:50]  COMMAND	POST 	 "/wd/

sync facebook contacts with google contacts?



send facebook message?



In [ ]:
function sendFacebookMessage(message) {
    return client
        .click('//*[contains(@aria-label, "Type a message")]')
//        .keys(['Control', 'A', 'NULL'])
        .keys('\uE127')
        .keys(message)
        .keys('\uE007')
        .catch(e => console.log(e))
}
module.exports = sendFacebookMessage;


connect add friends facebook?


In [5]:
var importer = require('../Core');
var glob = require('glob');
var path = require('path');
var fs = require('fs');

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var project = PROFILE_PATH + '/Conversations';

var regexToArray = (ex, str, i = 0) => {
    var co = []; var m;
    while ((m = ex.exec(str)) && co.push(m[i])) ;
    return co;
}
                                         
function addFacebookFriends() {
    // search posts
    var posts = glob.sync('**/*-posts-*', {cwd: project})
        .map(f => path.join(project, f));
    var friends = posts.reduce((acc, p) => {
        importer.streamJson(p, [true, {emitPath: true}], (match) => {
            [].concat(match.value).forEach(m => {
                const url = regexToArray(/facebook.com\/.*/ig, m);
                if(url && url[0]) {
                    acc.push(url[0].replace(/\?fref.*|\?ref.*|\?rc=.*|\?hc_location.*/ig, ''));
                }
            })
        });
        return acc;
    }, []);
    var unique = [];
    for(let i = 0; i < friends.length / 1000; i++) {
        unique = unique.concat(friends.slice(i * 1000, i * 1000 + 1000).filter((u, i, arr) => arr.indexOf(u) === i))
    }
    fs.writeFileSync(path.join(project, 'new-friends.json'), JSON.stringify(unique, null, 4));
    // TODO: diff file-system (abstract better?, code reuse?), selenium connect
    return unique;
}
module.exports = addFacebookFriends;

[Function: addFacebookFriends]

In [6]:

console.log(addFacebookFriends())


[ 'facebook.com/dadsrawesome/',
  'facebook.com/bouncycastles',
  'facebook.com/cbratton85',
  'facebook.com/theresantos',
  'facebook.com/alicia.welch.1671',
  'facebook.com/profile.php?id=17221941&hc_location=ufi',
  'facebook.com/profile.php?id=100009203532823&hc_location=ufi',
  'facebook.com/laura.calder.16',
  'facebook.com/kurt.scott.3720',
  'facebook.com/kristen.herndon.9',
  'facebook.com/rebecca.a.lee.9',
  'facebook.com/dana.bell.7796',
  'facebook.com/Tychvlo',
  'facebook.com/terry.mcmurrin.1',
  'facebook.com/micky.rupp',
  'facebook.com/steven.mcwilliams.378',
  'facebook.com/april.j.olson',
  'facebook.com/james.dopson.121',
  'facebook.com/megan.campbell.125760',
  'facebook.com/gemma.tenedero.1',
  'facebook.com/Jermboi',
  'facebook.com/dbs03',
  'facebook.com/maritsa.j.reynoso',
  'facebook.com/lauren.wilson.733',
  'facebook.com/william.faul.7',
  'facebook.com/profile.php?id=100009630318628&fref=ufi&rc=p',
  'facebook.com/amanda.keating.716',
  'facebook.com/khyl

tell a joke on facebook messenger?

run commands from facebook messenger?

